# LELEC2870 Machine Learning Project: Predicting a film's gross revenue

## Pauline Terlinden

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
import scipy

import torch
import torch.nn as nn
import torch.nn.functional as F
%matplotlib inline

In [2]:
X2 = pd.read_csv("X2.csv", index_col=0)
X2.reset_index(drop=True, inplace=True)

X2.info()

X2.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1518 entries, 0 to 1517
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            1518 non-null   object 
 1   img_url          1518 non-null   object 
 2   description      1518 non-null   object 
 3   ratings          1518 non-null   float64
 4   n_votes          1518 non-null   float64
 5   is_adult         1518 non-null   int64  
 6   production_year  1518 non-null   int64  
 7   runtime          1518 non-null   object 
 8   genres           1518 non-null   object 
 9   release_year     1518 non-null   float64
 10  studio           1518 non-null   object 
 11  img_embeddings   1518 non-null   object 
 12  text_embeddings  1518 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 154.3+ KB


,title,img_url,description,ratings,n_votes,is_adult,production_year,runtime,genres,release_year,studio,img_embeddings,text_embeddings
0,My Life Without Me,https://m.media-amazon.com/images/M/MV5BNTkxNW...,My Life Without Me: Directed by Isabel Coixet....,7.5,24256.0,0,2003,106,"Drama,Romance",2003.0,SPC,"[0.2698399, 3.1845033, 0.13669977, 0.14299123,...","[-0.62699246, 0.3949766, 0.9997058, -0.9845231..."
1,Fist Fight,https://m.media-amazon.com/images/M/MV5BZjg5Nj...,Fist Fight: Directed by Robert Breer. An autob...,5.9,48.0,0,1964,11,Short,2017.0,WB (NL),"[0.4279979, 0.82890534, 1.0090543, 0.17800629,...","[-0.78522605, 0.46340582, 0.9999196, -0.991904..."
2,Ballerina,https://m.media-amazon.com/images/M/MV5BMzJmMj...,Ballerina: Directed by Harold D. Schuster. Wit...,6.3,6.0,0,1957,30,"Crime,Drama",2009.0,FRun,"[0.3227182, 4.6717577, 0.24727274, 0.30942556,...","[-0.51733744, 0.3527469, 0.9993828, -0.9729327..."
3,Hidalgo,https://m.media-amazon.com/images/M/MV5BM2MxZG...,Hidalgo: Directed by Paul Stanley. With James ...,7.9,80.0,0,1972,60,Western,2004.0,BV,"[0.011863205, 1.0377717, 0.6964849, 0.27411973...","[-0.52853703, 0.24114524, 0.998973, -0.9659761..."
4,Frank & Jesse,https://m.media-amazon.com/images/M/MV5BMjA0MD...,Frank & Jesse: Directed by Robert Boris. With ...,5.8,1359.0,0,1994,105,"Biography,Western",1994.0,Trim.,"[0.18395236, 2.0983813, 0.29292667, 0.59968936...","[-0.6487137, 0.4033376, 0.999519, -0.97546834,..."
5,Half Nelson,https://m.media-amazon.com/images/M/MV5BMTk2ND...,Half Nelson: Directed by Barry Crane. With Bil...,6.6,96.0,0,1981,48,"Action,Adventure,Drama",2006.0,Think,"[0.008287275, 0.5699366, 0.1108259, 0.01168934...","[-0.64981234, 0.35286474, 0.99949956, -0.97383..."
6,Head Above Water,https://m.media-amazon.com/images/M/MV5BNmYwYm...,Head Above Water: Directed by Jim Wilson. With...,6.0,7348.0,0,1996,92,"Comedy,Thriller",1997.0,FL,"[0.026730772, 0.7417417, 0.58168155, 0.4150187...","[-0.5898534, 0.3410388, 0.99950737, -0.9755165..."
7,Swim Team,https://m.media-amazon.com/images/M/MV5BMjE0NT...,Swim Team: Directed by James Polakof. With Jam...,4.2,60.0,0,1979,75,Comedy,2017.0,Argo.,"[0.2939257, 0.3488631, 0.2870257, 0.022705443,...","[-0.75996625, 0.44631857, 0.9998441, -0.982079..."
8,Hadh Kar Di Aapne,https://m.media-amazon.com/images/M/MV5BZjc3MD...,Hadh Kar Di Aapne: Directed by Manoj Agrawal. ...,5.3,1713.0,0,2000,134,"Comedy,Romance",2000.0,Eros,"[0.6208895, 2.201567, 0.7864221, 0.2962354, 0....","[-0.6034175, 0.3644132, 0.9990996, -0.95884925..."
9,Looking for Mr. Goodbar,https://m.media-amazon.com/images/M/MV5BYzdjND...,Looking for Mr. Goodbar: Directed by Richard B...,6.7,6721.0,0,1977,136,Drama,1977.0,Par.,"[0.14248441, 1.5444763, 0.15048383, 0.34670386...","[-0.6973994, 0.39445153, 0.999695, -0.9841449,..."


# 1. Data preprocessing

## 1.1 Deletion of irrelevant variables and missing values

In [3]:
# Dropping title (irrelevant), img_url and description (unnecessary since we have the associated embeddings), is_adult (only 0's)
df = X2.drop(columns=['img_url', 'description', 'title', 'is_adult'])

#print(len(X2) - X2.count())

df.dropna(inplace=True)
# Watch out : for 'runtime', no explicit NaN's, but some values are '\N'
df["runtime"] = df["runtime"].replace('\\N', 0)

print(df.shape)

(1518, 9)


## 1.2 Basic manipulation of some features to make them more usable

In [4]:
print(df.dtypes)

ratings            float64
n_votes            float64
production_year      int64
runtime             object
genres              object
release_year       float64
studio              object
img_embeddings      object
text_embeddings     object
dtype: object


### 1.2.1 Replacing the genres column with one binary value for each possible genre

In [5]:
genres = set()
for string in df['genres']:
    genres_loc = string.split(",")
    for genre in genres_loc:
        if genre not in genres and genre != '\\N':
            genres.add(genre)
            
print(genres)
genres = list(genres)
for genre in genres:
    df[genre] = np.where(df.genres.str.contains(genre), 1, 0)


df.drop(columns=['genres'], inplace=True)
df.head(10)

{'Romance', 'Musical', 'Drama', 'Fantasy', 'Sport', 'Family', 'Game-Show', 'Short', 'Action', 'Comedy', 'Biography', 'Crime', 'Music', 'Western', 'Thriller', 'Animation', 'Horror', 'History', 'War', 'Documentary', 'Mystery', 'Sci-Fi', 'Adventure', 'Reality-TV'}


,ratings,n_votes,production_year,runtime,release_year,studio,img_embeddings,text_embeddings,Romance,Musical,...,Thriller,Animation,Horror,History,War,Documentary,Mystery,Sci-Fi,Adventure,Reality-TV
0,7.5,24256.0,2003,106,2003.0,SPC,"[0.2698399, 3.1845033, 0.13669977, 0.14299123,...","[-0.62699246, 0.3949766, 0.9997058, -0.9845231...",1,0,...,0,0,0,0,0,0,0,0,0,0
1,5.9,48.0,1964,11,2017.0,WB (NL),"[0.4279979, 0.82890534, 1.0090543, 0.17800629,...","[-0.78522605, 0.46340582, 0.9999196, -0.991904...",0,0,...,0,0,0,0,0,0,0,0,0,0
2,6.3,6.0,1957,30,2009.0,FRun,"[0.3227182, 4.6717577, 0.24727274, 0.30942556,...","[-0.51733744, 0.3527469, 0.9993828, -0.9729327...",0,0,...,0,0,0,0,0,0,0,0,0,0
3,7.9,80.0,1972,60,2004.0,BV,"[0.011863205, 1.0377717, 0.6964849, 0.27411973...","[-0.52853703, 0.24114524, 0.998973, -0.9659761...",0,0,...,0,0,0,0,0,0,0,0,0,0
4,5.8,1359.0,1994,105,1994.0,Trim.,"[0.18395236, 2.0983813, 0.29292667, 0.59968936...","[-0.6487137, 0.4033376, 0.999519, -0.97546834,...",0,0,...,0,0,0,0,0,0,0,0,0,0
5,6.6,96.0,1981,48,2006.0,Think,"[0.008287275, 0.5699366, 0.1108259, 0.01168934...","[-0.64981234, 0.35286474, 0.99949956, -0.97383...",0,0,...,0,0,0,0,0,0,0,0,1,0
6,6.0,7348.0,1996,92,1997.0,FL,"[0.026730772, 0.7417417, 0.58168155, 0.4150187...","[-0.5898534, 0.3410388, 0.99950737, -0.9755165...",0,0,...,1,0,0,0,0,0,0,0,0,0
7,4.2,60.0,1979,75,2017.0,Argo.,"[0.2939257, 0.3488631, 0.2870257, 0.022705443,...","[-0.75996625, 0.44631857, 0.9998441, -0.982079...",0,0,...,0,0,0,0,0,0,0,0,0,0
8,5.3,1713.0,2000,134,2000.0,Eros,"[0.6208895, 2.201567, 0.7864221, 0.2962354, 0....","[-0.6034175, 0.3644132, 0.9990996, -0.95884925...",1,0,...,0,0,0,0,0,0,0,0,0,0
9,6.7,6721.0,1977,136,1977.0,Par.,"[0.14248441, 1.5444763, 0.15048383, 0.34670386...","[-0.6973994, 0.39445153, 0.999695, -0.9841449,...",0,0,...,0,0,0,0,0,0,0,0,0,0


### 1.2.2 Replacing 'studio' with a 'studio_pop' variable
As there are 498 different studios listed in this dataset, we deem it inefficient to create a dummy variable for every one of them. The main way in which we find it logical for a studio to influence a movie's gross revenue is through its popularity, i.e. how many movies it has produced in this dataset. An alternative in the same vein could have been to create dummy variables only for the most popular studios.

In [6]:
popularities = df['studio'].value_counts()   # This function yields exactly what we want !

df['studio_pop'] = df['studio'].map(popularities)

df.drop(columns=['studio'], inplace=True)
df.head()

,ratings,n_votes,production_year,runtime,release_year,img_embeddings,text_embeddings,Romance,Musical,Drama,...,Animation,Horror,History,War,Documentary,Mystery,Sci-Fi,Adventure,Reality-TV,studio_pop
0,7.5,24256.0,2003,106,2003.0,"[0.2698399, 3.1845033, 0.13669977, 0.14299123,...","[-0.62699246, 0.3949766, 0.9997058, -0.9845231...",1,0,1,...,0,0,0,0,0,0,0,0,0,24
1,5.9,48.0,1964,11,2017.0,"[0.4279979, 0.82890534, 1.0090543, 0.17800629,...","[-0.78522605, 0.46340582, 0.9999196, -0.991904...",0,0,0,...,0,0,0,0,0,0,0,0,0,5
2,6.3,6.0,1957,30,2009.0,"[0.3227182, 4.6717577, 0.24727274, 0.30942556,...","[-0.51733744, 0.3527469, 0.9993828, -0.9729327...",0,0,1,...,0,0,0,0,0,0,0,0,0,18
3,7.9,80.0,1972,60,2004.0,"[0.011863205, 1.0377717, 0.6964849, 0.27411973...","[-0.52853703, 0.24114524, 0.998973, -0.9659761...",0,0,0,...,0,0,0,0,0,0,0,0,0,52
4,5.8,1359.0,1994,105,1994.0,"[0.18395236, 2.0983813, 0.29292667, 0.59968936...","[-0.6487137, 0.4033376, 0.999519, -0.97546834,...",0,0,0,...,0,0,0,0,0,0,0,0,0,5


### 1.2.3 Manipulation of the 'production_year' and 'release_year' variables
We discard the production year, as this year sometimes largely predates the release year : indeed, the movie might have taken a long time to finally be released in the US. Furthermore, the release year may affect a movie's gross revenue through economical events influencing the attendance of cinemas. We consider this to be far less true for the production year, as the economy of that year only (potentially) affected the production, not the public which ultimately is the one generating the gross revenue.
Finally, as a year is not a quantitative variable, we "quantitize" it by creating a variable "years_relased", counting the number of years that have passed since the movie's release.

In [7]:
df.drop(columns=['production_year'], inplace=True)

df['years_released'] = df['release_year'].map(lambda x : 2022-x)
df.drop(columns=['release_year'], inplace=True)
df.head()

,ratings,n_votes,runtime,img_embeddings,text_embeddings,Romance,Musical,Drama,Fantasy,Sport,...,Horror,History,War,Documentary,Mystery,Sci-Fi,Adventure,Reality-TV,studio_pop,years_released
0,7.5,24256.0,106,"[0.2698399, 3.1845033, 0.13669977, 0.14299123,...","[-0.62699246, 0.3949766, 0.9997058, -0.9845231...",1,0,1,0,0,...,0,0,0,0,0,0,0,0,24,19.0
1,5.9,48.0,11,"[0.4279979, 0.82890534, 1.0090543, 0.17800629,...","[-0.78522605, 0.46340582, 0.9999196, -0.991904...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,5.0
2,6.3,6.0,30,"[0.3227182, 4.6717577, 0.24727274, 0.30942556,...","[-0.51733744, 0.3527469, 0.9993828, -0.9729327...",0,0,1,0,0,...,0,0,0,0,0,0,0,0,18,13.0
3,7.9,80.0,60,"[0.011863205, 1.0377717, 0.6964849, 0.27411973...","[-0.52853703, 0.24114524, 0.998973, -0.9659761...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,52,18.0
4,5.8,1359.0,105,"[0.18395236, 2.0983813, 0.29292667, 0.59968936...","[-0.6487137, 0.4033376, 0.999519, -0.97546834,...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,28.0


### 1.2.4 Converting the 'runtime' entries to floats
Indeed, now that the \N are removed, all of them are floats encoded as strings

In [8]:
df['runtime'] = df['runtime'].map(lambda x : float(x))

### 1.2.5 Handling the image embeddings

In [9]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

img_embeddings_list = ['img_' + str(i) for i in range(2048)]

#print(df['img_embeddings'].map(lambda x : float((x[1:-1].split(","))[0])).head())

for i in range(2048):
    new_col = img_embeddings_list[i]
    df[new_col] = df['img_embeddings'].map(lambda x : float((x[1:-1].split(","))[i]))

df.drop(columns=['img_embeddings'], inplace=True)
df.head()

,ratings,n_votes,runtime,text_embeddings,Romance,Musical,Drama,Fantasy,Sport,Family,...,img_2038,img_2039,img_2040,img_2041,img_2042,img_2043,img_2044,img_2045,img_2046,img_2047
0,7.5,24256.0,106.0,"[-0.62699246, 0.3949766, 0.9997058, -0.9845231...",1,0,1,0,0,0,...,0.964871,0.313221,0.482249,0.309053,0.051615,0.181580,0.086231,0.355150,0.535751,0.398309
1,5.9,48.0,11.0,"[-0.78522605, 0.46340582, 0.9999196, -0.991904...",0,0,0,0,0,0,...,0.835874,0.175062,1.970724,0.549192,1.993706,0.169411,0.619817,0.259562,0.000390,0.026740
2,6.3,6.0,30.0,"[-0.51733744, 0.3527469, 0.9993828, -0.9729327...",0,0,1,0,0,0,...,0.974968,0.280686,0.028238,0.655622,0.173037,0.141024,0.101441,0.021153,0.672859,0.255673
3,7.9,80.0,60.0,"[-0.52853703, 0.24114524, 0.998973, -0.9659761...",0,0,0,0,0,0,...,0.007504,0.192608,0.171784,0.265927,0.563667,0.373161,0.628449,0.497885,0.322534,0.117319
4,5.8,1359.0,105.0,"[-0.6487137, 0.4033376, 0.999519, -0.97546834,...",0,0,0,0,0,0,...,1.138478,0.495138,1.266172,0.208854,0.961946,0.093299,0.246985,0.253747,0.088144,0.202934


### 1.2.6 Handling the text embeddings

In [10]:
text_embeddings_list = ['text_' + str(i) for i in range(768)]

for i in range(768):
    new_col = text_embeddings_list[i]
    df[new_col] = df['text_embeddings'].map(lambda x : float((x[1:-1].split(","))[i]))

df.drop(columns=['text_embeddings'], inplace=True)
df.head()

,ratings,n_votes,runtime,Romance,Musical,Drama,Fantasy,Sport,Family,Game-Show,...,text_758,text_759,text_760,text_761,text_762,text_763,text_764,text_765,text_766,text_767
0,7.5,24256.0,106.0,1,0,1,0,0,0,0,...,0.663820,-0.091275,0.457304,-0.391982,-0.397648,-0.350135,-0.076778,0.999912,-0.499187,0.972007
1,5.9,48.0,11.0,0,0,0,0,0,0,0,...,0.751991,0.104360,0.117178,-0.165869,-0.142738,0.049725,-0.263748,0.999960,-0.219024,0.981100
2,6.3,6.0,30.0,0,0,1,0,0,0,0,...,0.551550,-0.417442,0.580056,-0.656573,-0.472116,-0.505535,-0.086493,0.999815,-0.679789,0.965875
3,7.9,80.0,60.0,0,0,0,0,0,0,0,...,0.450442,-0.254761,0.560075,-0.624198,-0.503023,-0.568241,-0.025172,0.999743,-0.610655,0.932034
4,5.8,1359.0,105.0,0,0,0,0,0,0,0,...,0.516519,-0.050863,0.269495,-0.467202,-0.391554,-0.509108,-0.164548,0.999884,-0.521062,0.967667


In [11]:
df_after_embeddings = df.copy()   # to not have to rerun everything

## 1.3 Balance of influence: scaling

In [12]:
scaler = MinMaxScaler()   # StandardScaler might be required for some algorithms (ex: SVM) which require normality of the inputs

df = scaler.fit_transform(df)

# Convert back to pandas dataframe !
df = pd.DataFrame(df, columns=df_after_embeddings.columns)

df.head()

,ratings,n_votes,runtime,Romance,Musical,Drama,Fantasy,Sport,Family,Game-Show,...,text_758,text_759,text_760,text_761,text_762,text_763,text_764,text_765,text_766,text_767
0,0.7000,1.023556e-02,0.187279,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.815901,0.530562,0.729078,0.477977,0.391629,0.381623,0.409864,0.973912,0.313695,0.835186
1,0.5000,1.814890e-05,0.019435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.893117,0.725455,0.446939,0.693282,0.711379,0.709434,0.122062,0.989799,0.502229,0.898873
2,0.5500,4.220675e-07,0.053004,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.717581,0.205632,0.830904,0.226032,0.298218,0.254223,0.394910,0.942044,0.192159,0.792240
3,0.7500,3.165507e-05,0.106007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.629035,0.367697,0.814329,0.256860,0.259450,0.202816,0.489301,0.918461,0.238683,0.555219
4,0.4875,5.714795e-04,0.185512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.686902,0.570821,0.573288,0.406352,0.399273,0.251294,0.274760,0.964685,0.298973,0.804792


## 2.1 Applying PCA to the embeddings
This step is not actually feature *selection*, as we create new features based on the embeddings. Intuitivel, we project these embeddings into lower dimensional spaces, to make further computations faster.

In [13]:
from sklearn.decomposition import PCA

# Apply PCA to image embeddings
df_img = df.filter(regex=("img_.*"))

pca_img = PCA(n_components=0.8)
df_img = pca_img.fit_transform(df_img)

PCA_img_list = ['PCA_img_' + str(i) for i in range(pca_img.n_components_)]
df_img = pd.DataFrame(df_img, columns=PCA_img_list)

df.drop(list(df.filter(regex = 'img_.*')), axis = 1, inplace = True)
df = pd.concat([df, df_img], axis=1)


# Apply PCA to text embeddings
df_text = df.filter(regex=("text_.*"))

pca_text = PCA(n_components=0.95)
df_text = pca_text.fit_transform(df_text)

print(pca_text.n_components_)
PCA_text_list = ['PCA_text_' + str(i) for i in range(pca_text.n_components_)]
df_text = pd.DataFrame(df_text, columns=PCA_text_list)

df.drop(list(df.filter(regex = 'text_.*')), axis = 1, inplace = True)
df = pd.concat([df, df_text], axis=1)


df.head()

24


,ratings,n_votes,runtime,Romance,Musical,Drama,Fantasy,Sport,Family,Game-Show,...,PCA_text_14,PCA_text_15,PCA_text_16,PCA_text_17,PCA_text_18,PCA_text_19,PCA_text_20,PCA_text_21,PCA_text_22,PCA_text_23
0,0.7000,1.023556e-02,0.187279,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.185947,0.140147,-0.007608,-0.117475,-0.153722,0.056717,0.024998,-0.013394,0.058213,0.084241
1,0.5000,1.814890e-05,0.019435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.110904,0.469731,0.067939,-0.543848,-0.309169,0.046986,-0.035137,-0.043054,0.307078,-0.319627
2,0.5500,4.220675e-07,0.053004,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.130664,0.362059,-0.349610,0.142736,0.103428,-0.138595,-0.118504,0.143097,-0.095602,-0.516822
3,0.7500,3.165507e-05,0.106007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.317354,0.077573,0.054243,-0.223273,-0.144695,0.077987,-0.203574,0.040942,-0.037275,-0.055142
4,0.4875,5.714795e-04,0.185512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.159150,-0.064136,-0.183396,0.296490,-0.101797,0.034118,-0.348596,0.108428,-0.138929,0.196285


In [14]:
df.to_csv("X2_preprocessed.csv", index=False)

In [15]:
df.head()

,ratings,n_votes,runtime,Romance,Musical,Drama,Fantasy,Sport,Family,Game-Show,...,PCA_text_14,PCA_text_15,PCA_text_16,PCA_text_17,PCA_text_18,PCA_text_19,PCA_text_20,PCA_text_21,PCA_text_22,PCA_text_23
0,0.7000,1.023556e-02,0.187279,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.185947,0.140147,-0.007608,-0.117475,-0.153722,0.056717,0.024998,-0.013394,0.058213,0.084241
1,0.5000,1.814890e-05,0.019435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.110904,0.469731,0.067939,-0.543848,-0.309169,0.046986,-0.035137,-0.043054,0.307078,-0.319627
2,0.5500,4.220675e-07,0.053004,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.130664,0.362059,-0.349610,0.142736,0.103428,-0.138595,-0.118504,0.143097,-0.095602,-0.516822
3,0.7500,3.165507e-05,0.106007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.317354,0.077573,0.054243,-0.223273,-0.144695,0.077987,-0.203574,0.040942,-0.037275,-0.055142
4,0.4875,5.714795e-04,0.185512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.159150,-0.064136,-0.183396,0.296490,-0.101797,0.034118,-0.348596,0.108428,-0.138929,0.196285
